In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

import sys
sys.path.append('/Users/hzm/Desktop/YangTeng/code/venv/code/00.module')
import Proxy
import UA
import time

# = = = = = = = = = = = = = = = = = =

import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu.xlsx', header=0)
list_url = df_menu['Url'].to_list()

print('总数量：' + str(len(list_url)))
print()

df = pd.DataFrame()

# = = = = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =

        status = 'error'
        for _ in range(11):
            try:
                resp = requests.get(url, proxies=Proxy.get_Proxy_requests()).text

                # = = = = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                oe = ';'.join(html.xpath('//section[@id="productOE"]/descendant::*/th[@scope="row"]/text()'))

                # = = = = = = = = = = = = = = = = = =
                
                if len(html.xpath('//input[@id="hidfDetailApp"]/@value')) == 0:
                    vehicle_url = ''
                else:
                    vehicle_url = 'https://www.dormanproducts.com/' + html.xpath('//input[@id="hidfDetailApp"]/@value')[0]

                # = = = = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@id="imgProductMain"]/@src')[0]

                # = = = = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'OE_Numbers': oe,
                                         'Url': url,
                                         'vehicle_url': vehicle_url,
                                         'Src': src}])
                
                # = = = = = = = = = = = = = = = = = =
                
                list_row = html.xpath('//section[@id="productSpec"]/descendant::tr')
                for row in list_row:
                    df_temp[row.xpath('./th')[0].text.strip()[:-1].lower()] = row.xpath('./td')[0].text
                
                status = 'ok'
                
                break
            
            except:
                time.sleep(3)
                continue
                    
        # = = = = = = = = = = = = = = = = = =
        
        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余：' + str(work.qsize()))

# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('输出中...')
df.to_excel('./detail.xlsx', index=False)
print()
print('搞定！')

总数量：150

https://www.dormanproducts.com/p-131697-599-210xd.aspx?origin=keyword  <->  [ok] - 剩余：140
https://www.dormanproducts.com/p-76754-599-226.aspx?origin=keyword  <->  [ok] - 剩余：139
https://www.dormanproducts.com/p-131698-599-211xd.aspx?origin=keyword  <->  [ok] - 剩余：138
https://www.dormanproducts.com/p-66592-599-149.aspx?origin=keyword  <->  [ok] - 剩余：137
https://www.dormanproducts.com/p-82894-599-228.aspx?origin=keyword  <->  [ok] - 剩余：136
https://www.dormanproducts.com/p-72789-599-147.aspx?origin=keyword  <->  [ok] - 剩余：135
https://www.dormanproducts.com/p-128361-599-215.aspx?origin=keyword  <->  [ok] - 剩余：134
https://www.dormanproducts.com/p-76741-599-198.aspx?origin=keyword  <->  [ok] - 剩余：133
https://www.dormanproducts.com/p-76398-599-197.aspx?origin=keyword  <->  [ok] - 剩余：132
https://www.dormanproducts.com/p-53418-599-128.aspx?origin=keyword  <->  [ok] - 剩余：131
https://www.dormanproducts.com/p-65302-599-140.aspx?origin=keyword  <->  [ok] - 剩余：130
https://www.dormanproducts.

KeyboardInterrupt
2023-08-23T01:29:42Z


KeyboardInterrupt: 

https://www.dormanproducts.com/p-48281-599-019.aspx?origin=keyword  <->  [ok] - 剩余：117
https://www.dormanproducts.com/p-78185-599-206.aspx?origin=keyword  <->  [ok] - 剩余：116
